In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn.functional as F

from transformers import GPT2LMHeadModel, GPT2Tokenizer

from synth import CrossEntropyDifferential

/home/tuccio/synth/menv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda")

model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

cross_entropy_differential = CrossEntropyDifferential(model, tokenizer, device)

/home/tuccio/synth/menv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
Y = """ thing in the world, all the mozzarella and the tomato sauce on my margherita are amazing. 
My Italian friend Angelo told me that his favourite pizza is the quattroformaggi rossa with salsiccia. 
When I was a kid, I used to watch the pizza maker create his pizzas, he was from Romania but a very nice gentleman I have to say."""

prompt = f"""{Y}\n::INTRODUCE::"""
input = tokenizer(prompt, return_tensors='pt', padding=True).to(device)

outputs = torch.tensor([])
for i in range(3):
    torch.cuda.empty_cache()
    output = model.generate(
        input.input_ids,
        attention_mask=input.attention_mask,
        max_length=len(input.input_ids[0])+6,
        num_return_sequences=10,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=1
    )
    outputs = torch.cat((outputs, output.to(torch.device('cpu'))), dim=0)

results = []
for i in output:
    answ = tokenizer.decode(i[len(input.input_ids[0]):], skip_special_tokens=True)
    results.append((
        cross_entropy_differential(answ, Y, diff=True).item(),
        answ
        ))
best_result = min(results, key=lambda x: x[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


torch.Size([84, 50257])
torch.Size([84])
torch.Size([84, 50257])
torch.Size([84])
torch.Size([84, 50257])
torch.Size([84])
torch.Size([84, 50257])
torch.Size([84])
torch.Size([84, 50257])
torch.Size([84])
torch.Size([84, 50257])
torch.Size([84])
torch.Size([84, 50257])
torch.Size([84])
torch.Size([84, 50257])
torch.Size([84])
torch.Size([84, 50257])
torch.Size([84])
torch.Size([84, 50257])
torch.Size([84])


In [5]:
print(best_result)

(-0.1382145881652832, '\n*This is my favourite')


: 